In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH="../dat/textbooks/"
THRESHOLD = 2

In [3]:
df_de2 = pd.read_json(PATH+"de2.json")
df_de3 = pd.read_json(PATH+"de3.json")
adj_concepts = np.load(PATH+"concepts.npy")
adj_matrix = np.load(PATH+"adj.npy")

In [4]:
pruned = 0
total_deps = 0
for i, concept in enumerate(adj_concepts):
    delete_deps = set(adj_concepts[adj_matrix[i,:]<=10-THRESHOLD])
    proposed_deps = [d[3] for d in df_de3.loc[df_de3.concept==concept].iloc[0].dep_articles]
    #proposed_deps += [d for d in df_de2.loc[df_de2.concept==concept].iloc[0].dep_articles]
    pruned_deps = set(proposed_deps).intersection(delete_deps)
    total_deps += len(proposed_deps)
    if len(pruned_deps):
        pruned += len(pruned_deps)
        #print(f"on {concept} pruning " + str(pruned_deps)
        
print(f"{pruned}/{total_deps} are to be pruned")

279/6946 are to be pruned


In [5]:
def prune_deps(concept, deps):
    i = np.where(adj_concepts==concept)[0][0]
    delete_deps = set(adj_concepts[adj_matrix[i,:]<=10-THRESHOLD])
    return list(set(deps).difference(delete_deps))
    

In [6]:
df_concepts = pd.merge(df_de2, df_de3, on="concept", suffixes=("_de2","_de3"))
df_concepts['dep_articles_de3'] = df_concepts['dep_articles_de3'].apply(lambda x: [d[3] for d in x])
df_concepts['dep_articles'] = df_concepts['dep_articles_de2'] + df_concepts['dep_articles_de3']
df_concepts.dep_articles = df_concepts.apply(lambda r: prune_deps(r['concept'], [d for d in r['dep_articles']]), axis=1)

In [7]:
df_concepts.to_json(PATH+"graph.json")
df_concepts.to_csv(PATH+"df_concepts2.csv",index=False)